In [ ]:
pip install pandas

In [ ]:
### Step 1: Load and prep BLS Data
#Import packages
import os
import requests
import pandas as pd
import urllib.request as urllib
import numpy as np

#Notify user of load.
print("Loading data...")

#Load most recent U.Detail file from BEA Website
bea_udetail_raw = pd.read_excel('https://apps.bea.gov/industry/Release/XLS/UGdpxInd/GrossOutput.xlsx',
                                sheet_name = 'UGO305-A')

#Remove file header rows
#bea_udetail_raw.drop(bea_udetail_raw.iloc[:,3,].first_valid_index(), inplace=True)
bea_udetail_raw.drop(index = bea_udetail_raw.index[:7,],
                   axis = 0,
                   inplace = True)

bea_udetail_raw.drop(bea_udetail_raw.index[1])

#Rename Columns
bea_udetail_raw.columns = ['Row',
                           'BEA U.Detail Industry',
                           'Ignore',
                           '1997_Revenues',
                           '1998_Revenues',
                           '1999_Revenues',
                           '2000_Revenues',
                           '2001_Revenues',
                           '2002_Revenues',
                           '2003_Revenues',
                           '2004_Revenues',
                           '2005_Revenues',
                           '2006_Revenues',
                           '2007_Revenues',
                           '2008_Revenues',
                           '2009_Revenues',
                           '2010_Revenues',
                           '2011_Revenues',
                           '2012_Revenues',
                           '2013_Revenues',
                           '2014_Revenues',
                           '2015_Revenues',
                           '2016_Revenues',
                           '2017_Revenues',
                           '2018_Revenues',
                           '2019_Revenues',
                           '2020_Revenues',
                           '2021_Revenues',
                           '2022_Revenues',
                           '2023_Revenues',
                           '2024_Revenues',
                           '2025_Revenues']

#Remove row numbers and blank columns
bea_udetail = bea_udetail_raw.reindex(columns= [
    'Row',
    'BEA U.Detail Industry',
    '2019_Revenues',
    '2020_Revenues',
    '2021_Revenues',
    '2022_Revenues',
    '2023_Revenues',
    '2024_Revenues',
    '2025_Revenues'])

#Multiple values by millions to manage value display in BI visualization tool
bea_udetail.loc[:,'2019_Revenues'] *= 1000000
bea_udetail.loc[:,'2020_Revenues'] *= 1000000
bea_udetail.loc[:,'2021_Revenues'] *= 1000000
bea_udetail.loc[:,'2022_Revenues'] *= 1000000
bea_udetail.loc[:,'2023_Revenues'] *= 1000000
bea_udetail.loc[:,'2024_Revenues'] *= 1000000
bea_udetail.loc[:,'2025_Revenues'] *= 1000000

#Calculate the CAGRs
bea_udetail['2019_CAGR'] = ((bea_udetail['2019_Revenues'] - bea_udetail['2018_Revenues']) / bea_udetail['2018_Revenues'])
bea_udetail['2020_CAGR'] = ((bea_udetail['2020_Revenues'] - bea_udetail['2019_Revenues']) / bea_udetail['2019_Revenues'])
bea_udetail['2021_CAGR'] = ((bea_udetail['2021_Revenues'] - bea_udetail['2020_Revenues']) / bea_udetail['2020_Revenues'])
bea_udetail['2022_CAGR'] = ((bea_udetail['2022_Revenues'] - bea_udetail['2021_Revenues']) / bea_udetail['2021_Revenues'])
bea_udetail['2023_CAGR'] = ((bea_udetail['2023_Revenues'] - bea_udetail['2022_Revenues']) / bea_udetail['2022_Revenues'])
bea_udetail['2024_CAGR'] = ((bea_udetail['2024_Revenues'] - bea_udetail['2023_Revenues']) / bea_udetail['2023_Revenues'])
bea_udetail['2025_CAGR'] = ((bea_udetail['2025_Revenues'] - bea_udetail['2024_Revenues']) / bea_udetail['2024_Revenues'])

#Convert Row field to integers
bea_udetail['Row'] = bea_udetail['Row'].astype(int)

print("Load complete.")
print(f"Total rows: {len(bea_udetail)}")
display( bea_udetail.head(5) )

In [ ]:
### Step 3.1 Load Mapping File to bring in BEA Industry Group IDs
#Directory point to Mapping Files
%cd /Users/AndresJimenez/Documents/Jobs/Bank\ of\ America/Market\ Analysis\ Scripts/Mapping\ Files

# Load file
bea_map = pd.read_excel('BEA Concordence Map.xlsx', 
                          sheet_name = '2012-2017 BEA Concordance')
                          #encoding = 'unicode_escape')

#Rename columns - Pull original columns with: display(list(bea_map))
bea_map.columns =[
    'Sector_ID',
    'Sector_Name',
    'Summary_ID',
    'Summary_Name',
    'U.Summary_ID',
    'U.Summary_Name',
    'Industry_ID',
    'Industry_Name',
    'Notes',
    'NAICS_Digits',
    'BEA_Report_NAICS_Code',
    'BEA_Detail_Report_Industry_Value',
    'NAICS_2012_Description',
    '2017_NAICS_Code',
    '2017_NAICS_Description'
]

bea_map_short = bea_map.reindex(columns= [
    'Industry_ID',
    'BEA_Detail_Report_Industry_Value'
])

#Check files
display( bea_map_short.head(5) )
display(bea_map_short.dtypes)

In [ ]:
### STEP 3.3: Add BEA Industry Code

#Merge BEA Map and BEA Files
bea_full = pd.merge(bea_udetail,bea_map_short, how="left", 
                         left_on='BEA U.Detail Industry', right_on='BEA_Detail_Report_Industry_Value')

columns_to_check = list(bea_udetail.columns)
bea_full = bea_full.drop_duplicates(subset=columns_to_check, keep='first')

#Check files
display( bea_full.head(5) )
display(bea_full.dtypes)

In [ ]:
#Prepare file for export

#Rename Columns
bea_full.columns = [
    'Row',
    'BEA U.Detail Industry',
    '2017_Revenues',
    '2018_Revenues',
    '2019_Revenues',
    '2020_Revenues',
    '2021_Revenues',
    '2022_Revenues',
    '2018_CAGR',
    '2019_CAGR',
    '2020_CAGR',
    '2021_CAGR',
    '2022_CAGR',
    'BEA_Industry_ID',
    'Industry_Name'
]
bea_full = bea_full.reindex(columns= [
    'Row',
    'BEA_Industry_ID',
    'BEA U.Detail Industry',
    '2017_Revenues',
    '2018_Revenues',
    '2019_Revenues',
    '2020_Revenues',
    '2021_Revenues',
    '2022_Revenues',
    '2018_CAGR',
    '2019_CAGR',
    '2020_CAGR',
    '2021_CAGR',
    '2022_CAGR'
])

#Directory point to Mapping Files
%cd /Users/AndresJimenez/Documents/Jobs/Bank\ of\ America/Market\ Analysis\ Scripts/Bureau\ of\ Economic\ Analysis

#Export Files
bea_full.sort_values(by= ['Row']).to_excel('BEA_Gross Output_2017_2022.xlsx', index=True)